<a href="https://colab.research.google.com/github/maelleal/TCC_Recomendacao_INE/blob/main/Model_based_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U scikit-learn
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pandas
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [2]:
dataset = pandas.read_excel('/content/drive/My Drive/TCC - Sistema recomendação INE/Tabela para modelo medias v2.xlsx')
dataset = dataset.replace(["#N/A"], 0)
dataset_filtrado = dataset.drop(columns=['ID_UFSC', 'COD_MAT'])
dataset_filtrado = dataset_filtrado.replace(np.nan, 0)
dataset_filtrado = dataset_filtrado.round(2)
dataset_filtrado.head()
print(dataset_filtrado.dtypes)
print(dataset_filtrado)

MATRICULA       int64
ORIENTADOR      int64
COD_AREA      float64
MEDIA_NOTA    float64
Area1         float64
Area2         float64
Area3         float64
Area4         float64
Area5         float64
Area6         float64
Area7         float64
Area8         float64
Area9         float64
Area10        float64
Area11        float64
Area12        float64
Area13        float64
dtype: object
     MATRICULA  ORIENTADOR  COD_AREA  MEDIA_NOTA  Area1  Area2  Area3  Area4  \
0     11100878           5      10.0        9.50   8.84   7.60   6.90   7.50   
1     12200660           5      12.0       10.00   7.30   8.84   8.00   8.00   
2     12101248           5      10.0       10.00   0.00   0.00   0.00   0.00   
3      9232055          24       3.0        9.50   8.84   0.00   7.67   7.67   
4     12101265          12       3.0        9.67   0.00   0.00   0.00   0.00   
..         ...         ...       ...         ...    ...    ...    ...    ...   
309   15204125          13      10.0       10.00   7

In [3]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

'''
the user_item_matrix will look like this
|        | item 1 | ... | item m |
|--------|--------|-----|--------|
| user 1 | 3      | 0   | 0      |
| ...    | 0      | 4   | 5      |
| user n | 2      | 0   | 0      |
'''

# initial hyperparameter
epsilon = 1e-9
n_latent_factors = 10

# generate item lantent features
item_svd = TruncatedSVD(n_components = n_latent_factors)
item_features = item_svd.fit_transform(dataset_filtrado.transpose()) + epsilon

# generate user latent features
user_svd = TruncatedSVD(n_components = n_latent_factors)
user_features = user_svd.fit_transform(dataset_filtrado) + epsilon

In [4]:
!pip install -U scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 6.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633966 sha256=83cf8d16298b0909501128c985c3fcdcb54d1d7009bee02eda19e989046a215c
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [5]:
from surprise import SVD, accuracy
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import train_test_split

'''
rating dataframe will look like this
| user_id | item_id | rating          |
|---------|---------|-----------------|
| 1       | 1       | 5               |
| ...     | ...     | ...             |
| n       | m       | 3               |
'''

# preprocessing the data
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(dataset_filtrado[['MATRICULA','COD_AREA','MEDIA_NOTA']], reader)
train, test = train_test_split(data, test_size=.2, random_state=42)

# initial model
algo = SVD(random_state = 42)
algo.fit(train)
pred = algo.test(test)

# evaluate the rmse result of the prediction and ground thuth
accuracy.rmse(pred)

RMSE: 2.1855


2.185482523918402